# Flood ConvLSTM training

In [1]:
# Setup with GPU
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%load_ext autoreload
%autoreload 2
import tensorflow as tf
tf.config.list_physical_devices('GPU')

env: TF_FORCE_GPU_ALLOW_GROWTH=true


2024-07-12 01:53:42.364976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 01:53:42.365095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 01:53:42.442863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 01:53:42.725028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 01:53:47.962289: I external/local_xla/xla/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# Load the dataset form firestore
from usl_models.flood_ml.trainingdataset import IncrementalTrainDataGenerator

sim_names = ["Manhattan-config_v1%2FRainfall_Data_1.txt", "Manhattan-config_v1%2FRainfall_Data_2.txt"]
batch_size = 1 # On A100, use batch_size = 4

generator = IncrementalTrainDataGenerator()
dataset = generator.load_dataset_windowed(sim_names, batch_size=batch_size, max_chunks=8)


2024-07-12 01:53:48.358238: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 01:53:48.361852: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 01:53:48.364966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Initialize the model
from usl_models.flood_ml.model import FloodModel
from usl_models.flood_ml.model import Input
from usl_models.flood_ml.model_params import FloodModelParams

model = FloodModel(
    model_params=FloodModelParams(batch_size=batch_size))

In [4]:
# Fit the model to the dataset
model.fit(dataset, epochs=1)

In [5]:
dataset = generator.load_dataset_windowed(sim_names, batch_size=1, max_chunks=8)
input, labels = next(iter(dataset))
prediction = model.call(input)
prediction.shape, tf.math.reduce_max(prediction)

In [1]:
model.save_model('flood_model_tf213_20240711-104100')

In [7]:
# Test call_n
dataset = generator.load_dataset(sim_names, batch_size=1, max_chunks=1)
input, labels = next(iter(dataset))

(B, n, H, W) = labels.shape
predictions = model.call_n(input, n=n)
predictions.shape, tf.math.reduce_max(predictions)

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_2_3.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/2_3.npy


2024-07-12 01:53:53.465962: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.


Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


2024-07-12 01:53:54.273372: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.
2024-07-12 01:53:54.307598: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.
2024-07-12 01:53:54.498488: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


(TensorShape([1, 19, 1000, 1000]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011001954>)